# Issue Notebook Metadata

- Status: WIP
- Owner: hugolytics
- GitHub Issue: https://github.com/hugocool/FateForger/issues/7
- Issue Branch: issue/7-skeleton-pre-generation
- GitHub PR: https://github.com/hugocool/FateForger/pull/8
- Acceptance Criteria Reference: AC1-AC5 in tickets/skeleton_pre_generation.md and Issue #7
- Last Clean Run: 2026-02-13 16:57 UTC (markdown-only scaffold notebook)
- Environment: .venv / Python 3.11.9
- Repo Cleanliness Snapshot: dirty (`git status --porcelain`) at 2026-02-13 16:57 UTC (AGENTS/workflow updates in progress)


## Pairing Intake Record (Confirmed in Chat)

- Problem to solve: reduce ambiguity by forcing chat-first design alignment before notebook/code implementation.
- Constraints:
  - notebook-mode should be a real pairing surface, not only traceability metadata
  - human confirms direction before major coding
  - workflow remains GitHub-first for execution tracking
- Responsibility split:
  - Human: confirms problem framing, constraints, ownership boundaries, and chosen direction.
  - Agent: proposes options/tradeoffs/pseudocode, writes confirmed plan to notebook, executes modular implementation.
- Selected direction:
  - Add mandatory chat-first intake and design-options confirmation in workflow instructions.
  - Persist confirmed decisions in notebook before major coding.
- Open questions:
  - Preferred template verbosity for future `Pairing Intake Record` sections (short vs detailed).


## Design Options (Pre-Implementation Handshake)

### Option A (Recommended): Stage-local incremental extension
- Keep existing node boundaries and add focused fields/handlers.
- Tradeoff: Lowest risk and easiest review, but extends current flow complexity.
- Risks: state-flag drift unless tests are strong.
- Pseudocode outline:
  - capture stage -> if prerequisites ready: start skeleton pregen task
  - skeleton stage -> consume pregen result else fallback sync generation
  - review stage -> set pending_submit and emit confirm/cancel controls
  - confirm action -> submit, persist sync transaction, emit undo control
  - undo action -> undo last transaction, restore base snapshot, return refine

### Option B: Dedicated submission coordinator service
- Introduce service object coordinating submit/undo/pending state outside nodes.
- Tradeoff: clearer separation and future extensibility, but bigger refactor scope.
- Risks: touches more files/interfaces; higher regression risk for this ticket.
- Pseudocode outline:
  - coordinator.start_review(session) -> pending token + actions payload
  - coordinator.confirm(token) -> submit + store tx + post-submit payload
  - coordinator.undo(token) -> rollback + restore session snapshot

### Option C: Event-log-first state machine
- Model submit/undo as append-only events and derive session state from log.
- Tradeoff: strongest determinism/auditability, highest implementation cost now.
- Risks: over-engineering for current scope.
- Pseudocode outline:
  - append event(REVIEW_READY)
  - append event(SUBMIT_CONFIRMED, tx)
  - append event(UNDO_CONFIRMED, tx)
  - derive pending/committed/undone from replay

### Selected Option
- Selected: Option A
- Why: fits ticket scope AC1-AC5 with minimal churn and strong testability.


## Acceptance Criteria Checklist

- [x] AC1 Skeleton pre-generation in Stage 2 + Stage 3 consume/fallback path.
- [x] AC2 Confirm-before-submit review flow with Slack buttons.
- [x] AC3 Undo flow with post-submit undo button and handler.
- [x] AC4 Deterministic session memory transitions around submit/undo.
- [x] AC5 Integration tests for Slack button flows.
- [ ] Human verification in live Slack (required for `User-confirmed working`).


## Live Slack Verification (Manual)

Run this in the real Slack workspace and fill results:

1. Start a timeboxing session and reach Stage 5 review.
2. Confirm UI shows both buttons:
   - `✅ Submit to Calendar`
   - `↩️ Keep Editing`
3. Click `↩️ Keep Editing`.
   - Expected: pending submit clears and flow returns to Refine stage.
4. Click `✅ Submit to Calendar`.
   - Expected: calendar submit succeeds and message shows Undo button.
5. Click `↩️ Undo`.
   - Expected: rollback succeeds, base snapshot restored, flow returns to Refine stage.
6. End session and attempt Undo again.
   - Expected: rejected with ended-session guard.

Record evidence:
- Date/time:
- Workspace/channel/thread:
- Result: pass/fail
- Notes/screenshots links:


## Implementation Evidence

- Main implementation commit: `36182ba`.
- Ticket source: `tickets/skeleton_pre_generation.md`.
- Key module touchpoints:
  - `src/fateforger/agents/timeboxing/session.py`
  - `src/fateforger/agents/timeboxing/nodes/stage_capture_inputs_node.py`
  - `src/fateforger/agents/timeboxing/nodes/stage_skeleton_node.py`
  - `src/fateforger/agents/timeboxing/nodes/stage_review_commit_node.py`
  - `src/fateforger/agents/timeboxing/nodes/presenter_node.py`
  - `src/fateforger/slack_bot/handlers.py`
- Validation command sets executed:
  - `poetry run pytest tests/unit/test_timeboxing_skeleton_pre_generation.py tests/unit/test_timeboxing_submit_flow.py tests/unit/test_timeboxing_review_submit_prompt.py tests/integration/test_slack_timebox_buttons.py -q`
  - `poetry run pytest tests/unit/test_timeboxing_graphflow_state_machine.py tests/unit/test_phase4_rewiring.py tests/unit/test_slack_timeboxing_routing.py tests/unit/test_timeboxing_commit_skips_initial_extraction.py -q`


## Extraction Map (Notebook -> Artifacts)

| Notebook concern | Durable artifact destination |
| --- | --- |
| Skeleton pre-generation behavior | `src/fateforger/agents/timeboxing/nodes/stage_capture_inputs_node.py`, `src/fateforger/agents/timeboxing/nodes/stage_skeleton_node.py` |
| Confirm/undo submit workflow | `src/fateforger/agents/timeboxing/nodes/stage_review_commit_node.py`, `src/fateforger/agents/timeboxing/nodes/presenter_node.py`, `src/fateforger/slack_bot/handlers.py` |
| Session state persistence for submit/undo | `src/fateforger/agents/timeboxing/session.py` |
| Deterministic behavior checks | `tests/unit/test_timeboxing_skeleton_pre_generation.py`, `tests/unit/test_timeboxing_submit_flow.py`, `tests/unit/test_timeboxing_review_submit_prompt.py` |
| End-to-end Slack flow checks | `tests/integration/test_slack_timebox_buttons.py` |
| Human-readable execution trace | GitHub Issue #7 + PR #8 checkpoints |
| Product/knowledge context | Notion link from Issue/PR (if maintained) |


## Closeout / Remaining Notebook-Only Content

- This notebook is now a lightweight traceability scaffold (no duplicated production code cells).
- Remaining step before moving to `notebooks/DONE/`: perform live Slack verification and record confirmation date.
- After PR closure, keep this notebook only as a concise execution record or move to `notebooks/DONE/` per workflow decision.
